In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext()

In [2]:
# HDFS
hadoop_rdd = sc.hadoopFile("hdfs://localhost:9000/user/sebastian/sample_student_scores/input",\
                           "org.apache.hadoop.mapred.TextInputFormat",\
                           "org.apache.hadoop.io.Text",\
                           "org.apache.hadoop.io.LongWritable")
inputfile_RDD = hadoop_rdd.map(lambda pair : pair[1])

In [3]:
# non HDFS
#inputfile_RDD = sc.textFile("./sample_student_scores/input/*")

In [4]:
inputfile_RDD.collect()

['Kelly Math 88',
 'Kelly History 98',
 'Aston Math 93',
 'Aston History 93',
 'Leigh Math 75',
 'Kit Math 62',
 'Jewel History 79',
 'Mason Math 50',
 'Mason History 50',
 'Taylor Math 95',
 'Taylor History 95',
 'Hildred Math 99',
 'Hildred History 10',
 'Andy Math 45',
 'Brett Math 78',
 'Charlie Math 82',
 'Bernie Math 70',
 'Odell Math 33',
 'Robbie History 77',
 'Sammy History 88',
 'Sidney History 55',
 'Josh Math 25',
 'Josh History 35']

In [5]:
def removeIndexes(p):
    return p[1]
hadoop_rdd.map(removeIndexes).collect()

['Kelly Math 88',
 'Kelly History 98',
 'Aston Math 93',
 'Aston History 93',
 'Leigh Math 75',
 'Kit Math 62',
 'Jewel History 79',
 'Mason Math 50',
 'Mason History 50',
 'Taylor Math 95',
 'Taylor History 95',
 'Hildred Math 99',
 'Hildred History 10',
 'Andy Math 45',
 'Brett Math 78',
 'Charlie Math 82',
 'Bernie Math 70',
 'Odell Math 33',
 'Robbie History 77',
 'Sammy History 88',
 'Sidney History 55',
 'Josh Math 25',
 'Josh History 35']

In [6]:
import math

def scoresByClass(line):
    values = line.split();
    name = values[1]
    count = 1
    score = int(values[2])
    square_sum = score*score
    return (name, [count, score, square_sum])
def reduceScoresByClass(a, b):
    c = []
    c.append(a[0] + b[0])
    c.append(a[1] + b[1])
    c.append(a[2] + b[2])
    return c
def reduceScoresToMeanAndSD(p):
    print(p)
    mean = p[1][1]/p[1][0]
    sd = math.sqrt(p[1][2]/p[1][0]-mean*mean)
    cutoff = mean - sd
    return (p[0], cutoff)

In [7]:
cutoffs = inputfile_RDD.map(scoresByClass)\
                .reduceByKey(reduceScoresByClass)\
                .map(reduceScoresToMeanAndSD)\
                .collect()
cutoffs

[('Math', 45.61207359306223), ('History', 40.0679395675865)]

In [8]:
def getCutoff(class_name):
    for pair in cutoffs:
        if (pair[0] == class_name):
            return pair[1]

def scoresByName(line):
    values = line.split();
    name = values[0]
    class_name = values [1]
    score = int(values[2])
    return (name, [class_name, score])
def filterUsingCutoff(p):
    name = p[0]
    class_name = p[1][0]
    score = p[1][1]
    if (getCutoff(class_name) > score):
        return True
    else:
        return False
def removeScores(a):
    return (a[0], a[1][0])
def reduceClassesToName(a, b):
    return [a, b]

In [9]:
inputfile_RDD.map(scoresByName).filter(filterUsingCutoff).map(removeScores).reduceByKey(reduceClassesToName).collect()

[('Hildred', 'History'),
 ('Odell', 'Math'),
 ('Josh', ['Math', 'History']),
 ('Andy', 'Math')]